1. Basic code for machine translation (German -> English) using torchtext, Seq2Seq model using GRU.
2. There is a lot of scope of improvement like better architecture, better training and addressing overfitting.
3. You can also refer my another code which doesn't use torchtext at https://www.kaggle.com/code/priyankdl/machine-translation-seq-2-seq
4. You can also refer my another code which uses Bahdanau Attention mechanism but doesn't use torchtext  at https://www.kaggle.com/code/priyankdl/machine-translation-seq-2-seq-bahdanau-attention

In [ ]:
!pip install torchtext==0.15.2 torch==2.0.1

In [ ]:
!pip install 'portalocker>=2.0.0'
#you may require to restart the kernel

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader

In [ ]:
if torch.cuda.is_available():
    device=torch.device(type='cuda',index=0)
else:
    device=torch.device(type='cpu',index=0)
print(device)

In [ ]:
# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de' #German
TGT_LANGUAGE = 'en'

In [ ]:
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm 
#run a module spacy as a script and install the small version of spaCy’s 
#English language model, allowing you to process English text using spaCy for tasks 
#like tokenization, named entity recognition, and part-of-speech tagging.
!python -m spacy download de_core_news_sm

In [32]:
#setup the tokenizer for German and English
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')


#function to yield list of tokens for building the vocab
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    if language=='de':
        for data_sample in data_iter:
            yield de_tokenizer(data_sample[0])
    elif language=='en':
        for data_sample in data_iter:
            yield en_tokenizer(data_sample[1])
            
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

#build vocab for German
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

#torchtext.datasets.Multi30k(root: str = '.data', 
#split: Union[Tuple[str], str] = ('train', 'valid', 'test'), 
#language_pair: Tuple[str] = ('de', 'en')). Available options are (‘de’,’en’) and (‘en’, ‘de’)

#train: 29000, #valid: 1014, #test: 1000

#Returns DataPipe that yields tuple of source and target sentences (str,str)

vocab_de=build_vocab_from_iterator(yield_tokens(train_iter, 'de'),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_de.set_default_index(UNK_IDX)


#Now, build vocab for English
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_en=build_vocab_from_iterator(yield_tokens(train_iter, 'en'),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_en.set_default_index(UNK_IDX)

In [33]:
print(type(vocab_en))
print("English Vocab Length:",vocab_en.__len__())
print("German Vocab Length:",vocab_de.__len__())

<class 'torchtext.vocab.vocab.Vocab'>
English Vocab Length: 10837
German Vocab Length: 19214


In [34]:
#prepare separate batch for source sentence ids and target sentence ids
#insert <EOS> in source sentence
#insert <BOS> and <EOS> in target sentence
#pad sentences in a batch to same length
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        
        src_sample=src_sample.rstrip("\n") #string
        tgt_sample=tgt_sample.rstrip("\n")
        
        src_tokens=de_tokenizer(src_sample) #sentence/string to list of word tokens
        tgt_tokens=en_tokenizer(tgt_sample)
        
        src_ids=vocab_de(src_tokens) #from list of word tokens to list of ids
        tgt_ids=vocab_en(tgt_tokens)
        
        src_ids.append(EOS_IDX) #append <EOS> to list
        tgt_ids.append(EOS_IDX)
        
        tgt_ids.insert(0,BOS_IDX) #start with <BOS> in list
        
        src_tensor=torch.tensor(src_ids) #convert to tensor
        tgt_tensor=torch.tensor(tgt_ids)
        
        
        src_batch.append(src_tensor) #list of tensors
        tgt_batch.append(tgt_tensor)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True) #returns tensor
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    return src_batch, tgt_batch

In [35]:
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, dropout_p=0.1):
        super().__init__()
        self.e=nn.Embedding(input_size, embed_size)
        self.dropout=nn.Dropout(dropout_p)
        self.gru=nn.GRU(embed_size,hidden_size, batch_first=True)
    
    def forward(self,x):
        x=self.e(x)
        x=self.dropout(x)
        outputs, hidden=self.gru(x)
        return outputs, hidden

In [36]:
class Decoder(nn.Module):
    def __init__(self,output_size,embed_size,hidden_size):
        super().__init__()
        self.e=nn.Embedding(output_size,embed_size)
        self.relu=nn.ReLU()
        self.gru=nn.GRU(embed_size, hidden_size, batch_first=True)
        self.lin=nn.Linear(hidden_size,output_size)
        self.lsoftmax=nn.LogSoftmax(dim=-1)
    
    def forward(self,x,prev_hidden):
        x=self.e(x)
        x=self.relu(x)
        output,hidden=self.gru(x,prev_hidden)
        y=self.lin(output)
        y=self.lsoftmax(y)
        return y, hidden

In [37]:
def train_one_epoch():
    encoder.train()
    decoder.train()
    track_loss=0
    
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
    
    for i, (s_ids,t_ids) in enumerate(train_dataloader):
        
        s_ids=s_ids.to(device)
        t_ids=t_ids.to(device)
        encoder_outputs, encoder_hidden=encoder(s_ids)
        decoder_hidden=encoder_hidden
        yhats, decoder_hidden = decoder(t_ids[:,0:-1],decoder_hidden)
                    
        gt=t_ids[:,1:]
        
        yhats_reshaped=yhats.view(-1,yhats.shape[-1])
        
        gt=gt.reshape(-1)
        
        
        loss=loss_fn(yhats_reshaped,gt)
        track_loss+=loss.item()
        
        opte.zero_grad()
        optd.zero_grad()
        
        loss.backward()
        
        opte.step()
        optd.step()
        
    return track_loss/(i+1)
    

In [38]:
#eval loop (written assuming batch_size=1)
def eval_one_epoch(e,n_epochs):
    encoder.eval()
    decoder.eval()
    track_loss=0
    
    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=1, collate_fn=collate_fn)
    
    with torch.no_grad():
        for i, (s_ids,t_ids) in enumerate(val_dataloader):
            s_ids=s_ids.to(device) # [1, source sequence length]
            t_ids=t_ids.to(device)
            encoder_outputs, encoder_hidden=encoder(s_ids)
            decoder_hidden=encoder_hidden #n_dim=3
            input_id=t_ids[:,0] #shape is just [1]
            yhats=[]
            if e+1==n_epochs:
                pred_sentence=""
            for j in range(1,t_ids.shape[1]): #j starts from 1 #iterates for len(t_ids)-1 times as the last id is of <EOS>
                probs, decoder_hidden = decoder(input_id.unsqueeze(1),decoder_hidden) #need batch_size x seq_length
                yhats.append(probs)
                _,input_id=torch.topk(probs,1,dim=-1)
                input_id=input_id.squeeze(1,2) #still a tensor
                if e+1==n_epochs:                    
                    word=vocab_en.lookup_token(input_id.item()) #batch_size=1
                    pred_sentence+=word + " "
                if input_id.item() == 3: #batch_size=1 #Id 3 is <EOS>
                    break
                                
            if e+1==n_epochs:
                src_sentence_tokens=vocab_de.lookup_tokens(s_ids.tolist()[0])
                src_sentence=" ".join(src_sentence_tokens)
                gt_sentence_tokens=vocab_en.lookup_tokens(t_ids[:,1:].tolist()[0])
                gt_sentence=" ".join(gt_sentence_tokens)
                print("\n-----------------------------------")
                print("Source Sentence:",src_sentence)
                print("GT Sentence:",gt_sentence)
                print("Predicted Sentence:",pred_sentence)
            
            yhats_cat=torch.cat(yhats,dim=1)
            yhats_reshaped=yhats_cat.view(-1,yhats_cat.shape[-1])
            gt=t_ids[:,1:j+1] #shape is [1, target sequence length -1] #as <BOS> is not part of GT
            
            gt=gt.view(-1) #shape is [target sequence length -1]
            
            loss=loss_fn(yhats_reshaped,gt)
            track_loss+=loss.item()
            
        if e+1==n_epochs:    
            print("-----------------------------------")
        return track_loss/(i+1)   

In [39]:
embed_size=300
hidden_size=512
batch_size=32

encoder=Encoder(vocab_de.__len__(),embed_size,hidden_size).to(device) #translation-direction sensitive
decoder=Decoder(vocab_en.__len__(),embed_size,hidden_size).to(device) #translation-direction sensitive

loss_fn=nn.NLLLoss(ignore_index=1).to(device)
lr=0.001
opte=optim.Adam(params=encoder.parameters(), lr=lr, weight_decay=0.001)
optd=optim.Adam(params=decoder.parameters(), lr=lr, weight_decay=0.001)

n_epochs=20

for e in range(n_epochs):
    print("Epoch=",e+1, sep="", end=", ")
    print("Train Loss=", round(train_one_epoch(),4), sep="", end=", ")
    print("Eval Loss=",round(eval_one_epoch(e,n_epochs),4), sep="")

Epoch=1, Train Loss=4.0643, Eval Loss=5.9254
Epoch=2, Train Loss=3.5977, Eval Loss=5.7395
Epoch=3, Train Loss=3.5412, Eval Loss=5.8205
Epoch=4, Train Loss=3.5034, Eval Loss=5.8243
Epoch=5, Train Loss=3.4723, Eval Loss=5.8998
Epoch=6, Train Loss=3.4459, Eval Loss=6.016
Epoch=7, Train Loss=3.4224, Eval Loss=6.0756
Epoch=8, Train Loss=3.4003, Eval Loss=6.1872
Epoch=9, Train Loss=3.3801, Eval Loss=6.2614
Epoch=10, Train Loss=3.3636, Eval Loss=6.2465
Epoch=11, Train Loss=3.3494, Eval Loss=6.3544
Epoch=12, Train Loss=3.338, Eval Loss=6.3408
Epoch=13, Train Loss=3.3285, Eval Loss=6.4558
Epoch=14, Train Loss=3.3203, Eval Loss=6.5123
Epoch=15, Train Loss=3.3152, Eval Loss=6.5246
Epoch=16, Train Loss=3.3089, Eval Loss=6.5582
Epoch=17, Train Loss=3.3026, Eval Loss=6.5483
Epoch=18, Train Loss=3.2979, Eval Loss=6.6695
Epoch=19, Train Loss=3.2929, Eval Loss=6.7518
Epoch=20, Train Loss=3.2887, 
-----------------------------------
Source Sentence: Eine Gruppe von Männern lädt Baumwolle auf einen Lastw